In [1]:
import openai
from langchain import OpenAI, Wikipedia
from langchain.prompts import PromptTemplate
import csv
import pandas as pd
import os
import json
import time
import random
from tqdm import tqdm
from utils import read_data,shuffle_data,read_json,list2str
os.environ['OPENAI_API_KEY'] = 'Your-api-key'
from react_cls import Rank_agent,Knowledge_agent,Rethink_agent,Score_agent,EM
# 不同的LLM应该有不同的设置


# DDX
ddx_df = pd.read_csv('ddxplus.csv')
# 打乱'diag_set'列,'diag_set'列如['acute otitis media', 'urti', 'chagas']，先split，变成list，再shuffle
ddx_df = shuffle_data(ddx_df,colomn='Diag_Set')
# 读入知识库
with open('dis_responses.json', 'r', encoding='utf-8') as f:
    ddx_search = json.load(f)
    #把key转成小写，并且类似
    ddx_search = {k.lower(): v for k, v in ddx_search.items()}

In [2]:
K=5
CORRECT = 0
rank_agent = Rank_agent()
rethink_agent = Rethink_agent()
knowledge_agent = Knowledge_agent()
score_agent = Score_agent()
THRESHOLD = 0.5
# 遍历数据，进行问答
# 加入tqdm，显示进度条

for i in tqdm(range(0, len(ddx_df))):
    # 跳过第1，3，10个例子,相对的i是0，2，9
    if i == 0 or i == 2 or i == 9:
        continue
    ## 排序
    
    rank_str,rank_list = rank_agent.rank(ddx_df.iloc[i])
    print(rank_str)
    #迭代K次
    for k in range(K):
    
    ## Search+Rethink+Score
        ## Search
        #取出rank的第一个元素，即最高分的诊断
        
        top_diag = rank_list[0]
        top1_knowledge=Knowledge_agent.serach(top_diag)
        top1_knowledge = Knowledge_agent.summarize(top1_knowledge)
        print(top1_knowledge)
        ## Rethink
        rethink_rationale = Rethink_agent.rethink(ddx_df.iloc[i],rank_str,top1_knowledge)
        ## Score
        score = Score_agent.score(ddx_df.iloc[i],rank_str,rethink_rationale)
        if score > THRESHOLD:
            final_answer = top_diag
            CORRECT += EM(final_answer,ddx_df.iloc[i]['Diagnosis'])
            break
        else:
            #删除rank_list中的第一个元素
            rank_list.pop(0)
            rank_str = list2str(rank_list)
            #如果rank_list为空，说明没有找到合适的答案
            if len(rank_list) == 0:
                final_answer = 'None'
                break
            else:
                continue
            
ACC = CORRECT / (len(ddx_df)-3)
print(ACC)

d:\anaconda3\envs\chatGPT\lib\site-packages\langchain\llms\openai.py:170: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\anaconda3\envs\chatGPT\lib\site-packages\langchain\llms\openai.py:624: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
  2%|▏         | 2/100 [00:04<03:58,  2.44s/it]

1. tuberculosis; 2. viral pharyngitis; 3. urti; 4. bronchitis; 5. influenza; 6. possible nstemi / stemi; 7. acute laryngitis; 8. unstable angina; 9. epiglottitis; 10. stable angina


  4%|▍         | 4/100 [00:06<02:22,  1.48s/it]

1. urti; 2. cluster headache; 3. influenza; 4. anemia


  4%|▍         | 4/100 [00:08<03:31,  2.20s/it]


KeyboardInterrupt: 

In [5]:
print(rank_list)

['pulmonary embolism', 'localized edema', 'sle', 'chagas', 'anaphylaxis']
